In [15]:
from __future__ import division
import argparse
import os
import os.path as osp
import warnings
import xml.dom.minidom as minidom
import scipy.io as sio
import numpy as np
import json
import PIL
from PIL import Image
from tqdm import tqdm

In [4]:
imagenetHome = '../data/imagenet'
devkit_path = osp.join(imagenetHome,'devkit')

In [5]:
synsets_image = sio.loadmat(os.path.join(devkit_path, 'data', 'meta_det.mat'))
synsets_video = sio.loadmat(os.path.join(devkit_path, 'data', 'meta_vid.mat'))
classes_image = ('__background__',)
wnid_image = (0,)
classes = ('__background__',)
wnid = (0,)
for i in range(200):
    classes_image = classes_image + (synsets_image['synsets'][0][i][2][0],)
    wnid_image = wnid_image + (synsets_image['synsets'][0][i][1][0],)

for i in range(30):
    classes = classes + (synsets_video['synsets'][0][i][2][0],)
    wnid = wnid + (synsets_video['synsets'][0][i][1][0],)

wnid_to_ind_image = dict(zip(wnid_image, range(201)))
class_to_ind_image = dict(zip(classes_image, range(201)))

wnid_to_ind = dict(zip(wnid, range(31)))
class_to_ind = dict(zip(classes, range(31)))

#check for valid intersection between video and image classes
valid_image_flag = [0]*201

for i in range(1,201):
    if wnid_image[i] in wnid_to_ind:
        valid_image_flag[i] = 1

In [6]:
def get_data_from_tag(node, tag):
    return node.getElementsByTagName(tag)[0].childNodes[0].data
def has_tag(node, tag):
    return node.getElementsByTagName(tag).length>0

In [7]:
def parseImagenetAnnObject(objs, imgSize=None, has_trackid=False, allow_idx_to_trackid=True):
    num_objs = len(objs)
    if imgSize is not None:
        width, height = imgSize
    boxes = np.zeros((num_objs, 4), dtype=np.float32)
    gt_classes = np.zeros((num_objs), dtype=np.int64)
    if has_trackid:
        track_ids = np.zeros((num_objs), dtype=np.int64)
        
    # Load object bounding boxes into a data frame.
    for ix, obj in enumerate(objs):
        x1 = max(float(get_data_from_tag(obj, 'xmin')),0)
        y1 = max(float(get_data_from_tag(obj, 'ymin')),0)
        x2 = max(float(get_data_from_tag(obj, 'xmax')),0)
        y2 = max(float(get_data_from_tag(obj, 'ymax')),0)
        if imgSize is not None:
            # if image size is known, box bound can be trimmed.
            x1 = min(x1, width -1)
            y1 = min(y1, height-1)
            x2 = min(x2, width -1)
            y2 = min(y2, height-1)
        
        cls = wnid_to_ind[str(get_data_from_tag(obj, "name")).lower().strip()]
        boxes[ix, :] = [x1, y1, x2, y2]
        gt_classes[ix] = cls
        if has_trackid:
            if has_tag(obj, 'trackid'):
                track_ids[ix] = int(get_data_from_tag(obj, 'trackid'))
            elif allow_idx_to_trackid:
                track_ids[ix] = ix
            else:
                raise ValueError('obj does not have <trackid> tag.')
    
    if has_trackid is False:
        return {'bboxes':boxes.tolist(),
                'labels':gt_classes.tolist(),}
    else:
        return {'bboxes':boxes.tolist(),
                'labels':gt_classes.tolist(),
                'trackids':track_ids.tolist(),}
            

In [8]:
def load_imagenet_annotation(index_image, img_prefix, has_trackid=False):
    """
    Load image and bounding boxes info from txt files of imagenet.
    """
    # Get image infos
    filename = osp.join(img_prefix, index_image)
    assert os.path.exists(filename),'%s'%(filename)
    with Image.open(filename) as img:
        imgSize = img.size
    
    # Get ann infos
    filename = osp.join(img_prefix, index_image).replace('Data','Annotations').replace('.JPEG','.xml')
    index = filename
    assert os.path.exists(filename),'%s'%(filename)
    with open(filename) as f:
        data = minidom.parseString(f.read())

    objs = data.getElementsByTagName('object')
    num_objs = len(objs)
    #filter the objects not in video synsets.
    used_objs = []
    for id, obj in enumerate(objs):
        if str(get_data_from_tag(obj, "name")).lower().strip() in wnid_to_ind:
            used_objs.append(obj)
    objs = used_objs
    ann = parseImagenetAnnObject(objs,imgSize=img.size,has_trackid=has_trackid)
    return {
            'filename':index_image,
            'width':img.size[0],
            'height':img.size[1],
            'ann': ann
            }

In [9]:
def getCustomAnnByImagenetIndex(index,img_prefix):
    '''
    Annotation format:
    [
        {
            'filename': 'a.jpg',
            'width': 1280,
            'height': 720,
            'ann': {
                'bboxes': <np.ndarray> (n, 4),
                'labels': <np.ndarray> (n, ),
                'bboxes_ignore': <np.ndarray> (k, 4),
                'labels_ignore': <np.ndarray> (k, 4) (optional field)
            }
        },
        ...
    ]
    
    Args:
        index(str): relative file path.
        
    '''
    return load_imagenet_annotation(index,img_prefix)

In [10]:
def getCustomPairAnnByImagenetIndex(index1, index2, img_prefix):
    '''
    Annotation format:
    [
        {
            'filename1': 'a.jpg',
            'filename2': 'a.jpg',
            'width': 1280,
            'height': 720,
            'ann1': {
                'bboxes': <np.ndarray> (n, 4),
                'labels': <np.ndarray> (n, ),
                'trackids': <np.ndarray> (n, ),
                'bboxes_ignore': <np.ndarray> (k, 4),
                'labels_ignore': <np.ndarray> (k, 4) (optional field)
            }
            'ann2': {
                'bboxes': <np.ndarray> (n, 4),
                'labels': <np.ndarray> (n, ),
                'trackids': <np.ndarray> (n, ),
                'bboxes_ignore': <np.ndarray> (k, 4),
                'labels_ignore': <np.ndarray> (k, 4) (optional field)
            }
        },
        ...
    ]
    
    Args:
        index(str): relative file path.
        
    '''
    ann1 = load_imagenet_annotation(index1,img_prefix, has_trackid=True)
    ann2 = load_imagenet_annotation(index2,img_prefix, has_trackid=True)
    ann = {
            'filename1':ann1['filename'],
            'filename2':ann2['filename'],
            'width':ann1['width'],
            'height':ann1['height'],
            'ann1': ann1['ann'],
            'ann2': ann2['ann']
            }
    return ann

In [55]:
def ImagenetDETVIDSet2CustomAnn(imageDataPrefix, imagesetPath, cumstomStyleAnnPath):
    '''Convert Imagenet DETVID annotations to custom style annotations.
    
       The imagenet dataset is configured by 1 txt file in "ImageSets"
       directory(contain image index), while custom style annotations 
       have all anns in 1 file.
       
    Args:
        imagesetPath(str): imagenet imageset path.
        cumstomStyleAnnPath(str): the path to save annotations.
    '''
    assert cumstomStyleAnnPath.endswith('.json')
    file_indexes = []
    with open(imagesetPath,'r') as fr:
        lines = fr.readlines()
        for l in lines:
            items = l.split()
            file_indexes.append(items[0].replace('data/imagenet/ILSVRC/','')+'.JPEG')
    print('total number of files are %d.'%(len(file_indexes)))
    
    ann_list = []
    for i in tqdm(range(len(file_indexes))):
        index = file_indexes[i]
        ann_dict = getCustomAnnByImagenetIndex(index,imageDataPrefix)
        ann_list.append(ann_dict)
        
    with open(cumstomStyleAnnPath,'w') as fw:
        json.dump(ann_list, fw, indent=2)

In [161]:
def ImagenetVIDDirSet2CustomAnn(imageDataPrefix, imagesetPath, cumstomStyleAnnDirPath):
    assert osp.isdir(cumstomStyleAnnDirPath)
    dir_indexes = []
    with open(imagesetPath,'r') as fr:
        lines = fr.readlines()
        for l in lines:
            items = l.split()
            dir_path = items[0]
            if not 'ILSVRC' in osp.split(dir_path)[-1]:
                dir_path = osp.dirname(dir_path)
                assert 'ILSVRC' in osp.split(dir_path)[-1]
                if len(dir_indexes)==0 or dir_path!=dir_indexes[-1]:
                    dir_indexes.append(dir_path)
            else:
                assert 'ILSVRC' in osp.split(dir_path)[-1]
                dir_indexes.append(dir_path)
    print('total number of directories are %d.'%(len(dir_indexes)))
    
    for i in tqdm(range(len(dir_indexes))):
        ann_list = []
        index = dir_indexes[i]
        index_dir = osp.join(imageDataPrefix, index)
        fnames = os.listdir(index_dir)
        for j in range(len(fnames)):
            ann_dict = getCustomAnnByImagenetIndex(osp.join(index, '%06d.JPEG'%(j)),imageDataPrefix)
            ann_list.append(ann_dict)

        with open(osp.join(cumstomStyleAnnDirPath,'%06d.json'%(i)),'w') as fw:
            json.dump(ann_list, fw, indent=2)

In [121]:
def ImagenetVIDDirSet2CustomPairAnnSequential(imageDataPrefix, imagesetPath, cumstomStyleAnnPath, vidID=None):
    assert cumstomStyleAnnPath.endswith('.json')
    dir_indexes = []
    with open(imagesetPath,'r') as fr:
        lines = fr.readlines()
        for l in lines:
            items = l.split()
            dir_indexes.append(items[0])
    print('total number of directories are %d.'%(len(dir_indexes)))
    ann_list = []
    for i in tqdm(range(len(dir_indexes))):
        if vidID is not None and vidID!=i:
            continue
        index = dir_indexes[i]
        index_dir = osp.join(imageDataPrefix, index)
        fnames = os.listdir(index_dir)
        for j in range(0,len(fnames)-2):
            ann_dict = getCustomPairAnnByImagenetIndex(osp.join(index, '%06d.JPEG'%(j)), 
                                                       osp.join(index, '%06d.JPEG'%(j+1)), 
                                                       imageDataPrefix)
            ann_list.append(ann_dict)
    print('length of dataset:%d'%(len(ann_list)))
    with open(cumstomStyleAnnPath,'w') as fw:
        json.dump(ann_list, fw, indent=2)

In [118]:
def ImagenetVIDDirSet2CustomPairAnn(imageDataPrefix, imagesetPath, cumstomStyleAnnPath):
    assert cumstomStyleAnnPath.endswith('.json')
    dir_indexes = []
    with open(imagesetPath,'r') as fr:
        lines = fr.readlines()
        for l in lines:
            items = l.split()
            dir_indexes.append(items[0])
    print('total number of directories are %d.'%(len(dir_indexes)))
    ann_list = []
    interval = 10
    for i in tqdm(range(len(dir_indexes))):
        index = dir_indexes[i]
        index_dir = osp.join(imageDataPrefix, index)
        fnames = os.listdir(index_dir)
        for j in range(0,len(fnames),interval):
            for itv in range(int(interval/2)):
                if j+interval-itv-1<len(fnames):
                    ann_dict = getCustomPairAnnByImagenetIndex(osp.join(index, '%06d.JPEG'%(j+itv)), 
                                                    osp.join(index, '%06d.JPEG'%(j+interval-itv-1)), 
                                                    imageDataPrefix)
                    ann_list.append(ann_dict)
    with open(cumstomStyleAnnPath,'w') as fw:
        json.dump(ann_list, fw, indent=2)

In [96]:
def ImagenetDETSet2CustomPairAnn(imageDataPrefix, imagesetPath, cumstomStyleAnnPath):
    assert cumstomStyleAnnPath.endswith('.json')
    file_indexes = []
    with open(imagesetPath,'r') as fr:
        lines = fr.readlines()
        for l in lines:
            items = l.split()
            file_indexes.append(osp.join('Data','DET',items[0])+'.JPEG')
    print('total number of files are %d.'%(len(file_indexes)))
    
    ann_list = []
    for i in tqdm(range(len(file_indexes))):
        index = file_indexes[i]
        ann_dict = getCustomPairAnnByImagenetIndex(index,index,imageDataPrefix)
        ann_list.append(ann_dict)
        
    with open(cumstomStyleAnnPath,'w') as fw:
        json.dump(ann_list, fw, indent=2)

In [70]:
def ImagenetVIDSet2CustomAnn(imageDataPrefix, imagesetPath, cumstomStyleAnnPath):
    assert cumstomStyleAnnPath.endswith('.json')
    file_indexes = []
    with open(imagesetPath,'r') as fr:
        lines = fr.readlines()
        for l in lines:
            items = l.split()
            file_indexes.append(items[0]+'.JPEG')
    print('total number of files are %d.'%(len(file_indexes)))
    
    ann_list = []
    for i in tqdm(range(len(file_indexes))):
        index = file_indexes[i]
        ann_dict = getCustomAnnByImagenetIndex(index,imageDataPrefix)
        ann_list.append(ann_dict)
        
    with open(cumstomStyleAnnPath,'w') as fw:
        json.dump(ann_list, fw, indent=2)

In [71]:
def ImagenetDETSet2CustomAnn(imageDataPrefix, imagesetPath, cumstomStyleAnnPath):
    assert cumstomStyleAnnPath.endswith('.json')
    file_indexes = []
    with open(imagesetPath,'r') as fr:
        lines = fr.readlines()
        for l in lines:
            items = l.split()
            file_indexes.append(osp.join('Data','DET',items[0])+'.JPEG')
    print('total number of files are %d.'%(len(file_indexes)))
    
    ann_list = []
    for i in tqdm(range(len(file_indexes))):
        index = file_indexes[i]
        ann_dict = getCustomAnnByImagenetIndex(index,imageDataPrefix)
        ann_list.append(ann_dict)
        
    with open(cumstomStyleAnnPath,'w') as fw:
        json.dump(ann_list, fw, indent=2)

In [11]:
def ImagenetVIDSet2CustomAnn_every10frames(imageDataPrefix, imagesetPath, cumstomStyleAnnPath):
    assert cumstomStyleAnnPath.endswith('.json')
    file_indexes = []
    with open(imagesetPath,'r') as fr:
        lines = fr.readlines()
        for l in lines:
            items = l.split()
            idx = int(items[2])
            file_indexes.append(osp.join('Data','VID',items[0],'%06d'%(idx))+'.JPEG')
    print('total number of files are %d.'%(len(file_indexes)))
    
    ann_list = []
    for i in tqdm(range(len(file_indexes))):
        index = file_indexes[i]
        ann_dict = getCustomAnnByImagenetIndex(index,imageDataPrefix)
        ann_list.append(ann_dict)
        
    with open(cumstomStyleAnnPath,'w') as fw:
        json.dump(ann_list, fw, indent=2)

In [72]:
def ImagenetDETExperimentSet2CustomAnn(imageDataPrefix, imagesetPath, cumstomStyleAnnPath):
    assert cumstomStyleAnnPath.endswith('.json')
    file_indexes = []
    with open(imagesetPath,'r') as fr:
        lines = fr.readlines()
        for l in lines:
            items = l.split()
            file_indexes.append(osp.join('Data','DET',items[0])+'.JPEG')
    file_indexes = file_indexes[1000:1000+10]*20
    print('total number of files are %d.'%(len(file_indexes)))
    
    ann_list = []
    for i in tqdm(range(len(file_indexes))):
        index = file_indexes[i]
        ann_dict = getCustomAnnByImagenetIndex(index,imageDataPrefix)
        ann_list.append(ann_dict)
        
    with open(cumstomStyleAnnPath,'w') as fw:
        json.dump(ann_list, fw, indent=2)

#### DETVID

In [73]:
def main_DETVID_train():
    imagesetPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/trainr_DETVID.txt'
    cumstomStyleAnnPath = imagesetPath.replace('.txt','.json')
    ImagenetDETVIDSet2CustomAnn(imagenetHome, imagesetPath, cumstomStyleAnnPath)

#### DET

In [74]:
def main_DET_train():
    imagesetPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/DET_train_30classes.txt'
    cumstomStyleAnnPath = imagesetPath.replace('.txt','.json')
    ImagenetDETSet2CustomAnn(imagenetHome, imagesetPath,cumstomStyleAnnPath)

In [75]:
def main_DET_experiment_train():
    imagesetPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/DET_train_30classes.txt'
    cumstomStyleAnnPath = imagesetPath.replace('.txt','_experiment.json')
    ImagenetDETExperimentSet2CustomAnn(imagenetHome, imagesetPath,cumstomStyleAnnPath)

#### VID Val

In [76]:
def main_VID_val():
    imagesetPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/VID_val.txt'
    cumstomStyleAnnPath = imagesetPath.replace('.txt','.json')
    ImagenetVIDSet2CustomAnn(osp.join(imagenetHome,'Data/VID/val'), imagesetPath,cumstomStyleAnnPath)

#### VID Train

In [12]:
def main_VID_train():
    imagesetPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/VID_train_every10frames.txt'
    cumstomStyleAnnPath = imagesetPath.replace('.txt','.json')
    ImagenetVIDSet2CustomAnn_every10frames(imagenetHome, imagesetPath,cumstomStyleAnnPath)

In [162]:
def main_VID_train_multi_jsons():
    imagesetPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/VID_train.txt'
    cumstomStyleAnnPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/VID_jsons'
    ImagenetVIDDirSet2CustomAnn(osp.join(imagenetHome,'Data/VID/train'), imagesetPath,cumstomStyleAnnPath)

In [163]:
def main_VID_val_multi_jsons():
    imagesetPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/VID_val.txt'
    cumstomStyleAnnPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/VID_val_jsons'
    ImagenetVIDDirSet2CustomAnn(osp.join(imagenetHome,'Data/VID/val'), imagesetPath, cumstomStyleAnnPath)

#### VID Train Pair

In [149]:
def main_VID_train_pair():
    imagesetPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/VID_train.txt'
    cumstomStyleAnnPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/VID_train_pair.json'
    ImagenetVIDDirSet2CustomPairAnn(osp.join(imagenetHome,'Data/VID/train'), imagesetPath,cumstomStyleAnnPath)

In [150]:
def main_VID_train_pair_sequential():
    imagesetPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/VID_train.txt'
    cumstomStyleAnnPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/VID_train_pair_sequential.json'
    ImagenetVIDDirSet2CustomPairAnnSequential(osp.join(imagenetHome,'Data/VID/train'), imagesetPath,cumstomStyleAnnPath,vidID=1100)

#### DET Train Pair

In [97]:
def main_DET_train_pair():
    imagesetPath = '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Work/FastVOD/data/imagenet/ILSVRC/ImageSets/DET_train_30classes.txt'
    cumstomStyleAnnPath = imagesetPath.replace('.txt','_pair.json')
    ImagenetDETSet2CustomPairAnn(imagenetHome, imagesetPath,cumstomStyleAnnPath)

In [110]:
def combine_jsons(annfilelist,new_file_path):
    annlist = None
    for ann_file in annfilelist:
        ann = mmcv.load(ann_file)
        print('length of ann file: %d'%(len(ann)))
        if annlist is None:
            annlist=ann
        else:
            annlist = annlist+ann
    print('length of combined ann files: %d'%(len(annlist)))
    with open(new_file_path,'w') as fw:
        json.dump(ann, fw, indent=2)
    return annlist

In [14]:
if __name__=='__main__':
    #main_VID_train_pair()
    #main_VID_train_pair_sequential()
    #main_VID_val_multi_jsons()
    main_VID_train()
    #main_DET_train_pair()
    #combine_jsons(
    #    ['/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Dataset/ImageNet/ILSVRC2015/ImageSets/VID_train_pair.json',
    #     '/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Dataset/ImageNet/ILSVRC2015/ImageSets/DET_train_30classes_pair.json'
    #    ],new_file_path='/media/yelyu/18339a64-762e-4258-a609-c0851cd8163e/YeLyu/Dataset/ImageNet/ILSVRC2015/ImageSets/DETVID_train_pair.json')


  0%|          | 182/113666 [00:00<01:02, 1817.77it/s]

total number of files are 113666.



  2%|▏         | 2782/113666 [00:24<18:44, 98.63it/s]


  4%|▍         | 4365/113666 [00:45<24:15, 75.08it/s] 


  5%|▌         | 6023/113666 [01:06<19:41, 91.14it/s]


...

 94%|█████████▎| 106334/113666 [20:09<01:43, 70.96it/s]


 95%|█████████▌| 108114/113666 [20:31<02:29, 37.03it/s]


 97%|█████████▋| 110114/113666 [20:50<00:35, 98.70it/s]


 99%|█████████▊| 111986/113666 [21:12<00:14, 114.96it/s]


100%|██████████| 113666/113666 [21:28<00:00, 88.19it/s]